In [1]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()


import pystan

## Model Legend

* m6 = AZ
* m11 = EZ
* m9 = EFA
* m10 = EFA-C

In [2]:
## 500 particles - 300 data points
# check on tues 21.00
# 6 299
# 9 198
# 10 232
# 11 299



log_dirs = dict()
log_dirs['m6'] = 'log/fabian_real_muthen/20210906_192317_big5_m6/'
log_dirs['m11'] = 'log/fabian_real_muthen/20210906_192558_big5_m11/'
log_dirs['m10'] = 'log/fabian_real_muthen/20210906_192605_big5_m10/'
log_dirs['m9'] = 'log/fabian_real_muthen/20210906_192547_big5_m9/'


for i in [6, 9, 10, 11]:
    print(i, load_obj('t', log_dirs['m'+str(i)]))

6 299
9 299
10 299
11 299


In [2]:
## small experiment
## last
# 6 46
# 9 43
# 10 41
# 11 69


log_dirs = dict()
log_dirs['m6'] = 'log/fabian_real_muthen/20210829_201302_smallbig5_m6/'
log_dirs['m11'] = 'log/fabian_real_muthen/20210829_201308_smallbig5_m11/'
log_dirs['m10'] = 'log/fabian_real_muthen/20210829_201450_smallbig5_m10/'
log_dirs['m9'] = 'log/fabian_real_muthen/20210829_201305_smallbig5_m9/'



for i in [6, 9, 10, 11]:
    print(i, load_obj('t', log_dirs['m'+str(i)]))

6 199
9 199
10 199
11 199


In [14]:
## 1000 particles - longer timeout limit
# 6 415
# 9 290
# 10 333
# 11 676



log_dirs = dict()
log_dirs['m6'] = 'log/fabian_real_muthen/20210830_160211_big5_m6/'
log_dirs['m11'] = 'log/fabian_real_muthen/20210829_211924_big5_m11/'
log_dirs['m10'] = 'log/fabian_real_muthen/20210829_211839_big5_m10/'
log_dirs['m9'] = 'log/fabian_real_muthen/20210829_211918_big5_m9//'


for i in [6, 9, 10, 11]:
    print(i, load_obj('t', log_dirs['m'+str(i)]))

6 493
9 320
10 367
11 676


In [15]:
## 1000 particles - the runs timed out. 
## started at
# 6 228
# 9 143
# 10 162
# 11 392


log_dirs = dict()
log_dirs['m6'] = 'log/fabian_real_muthen/20210828_181350_big5_m6/'
log_dirs['m11'] = 'log/fabian_real_muthen/20210828_181300_big5_m11/'
log_dirs['m10'] = 'log/fabian_real_muthen/20210828_181359_big5_m10/'
log_dirs['m9'] = 'log/fabian_real_muthen/20210828_181357_big5_m9/'


for i in [6,9,10, 11]:
    print(i, load_obj('t', log_dirs['m'+str(i)]))

6 405
9 249
10 282
11 676


## Load Model Evidence

In [16]:
log_dirs = dict()
log_dirs['m6'] = 'log/fabian_real_muthen/20210829_201302_smallbig5_m6/'
log_dirs['m11'] = 'log/fabian_real_muthen/20210829_201308_smallbig5_m11/'
log_dirs['m10'] = 'log/fabian_real_muthen/20210829_201450_smallbig5_m10/'
log_dirs['m9'] = 'log/fabian_real_muthen/20210829_201305_smallbig5_m9/'
model_nums=[6, 9, 10, 11]

# load existing results
bf = dict()
for model in model_nums:
    bf['m'+str(model)] = np.cumsum(load_obj('log_lklhds', log_dirs['m'+str(model)])[30:])


In [17]:
bfdf = pd.DataFrame.from_dict(bf)
bfdf['t']  = np.arange(len(bfdf))
for i in model_nums:
    bfdf['m6/m'+str(i)] = bfdf['m6']-bfdf['m'+str(i)]
bfdf.drop(['m6', 'm11', 'm10', 'm9'], axis=1, inplace=True)
bfdf = bfdf.melt(id_vars='t', var_name='model')


In [18]:
width = 600
height = 400
c = alt.Chart(bfdf).mark_line(
    strokeWidth = 1,
    ).encode(
    alt.X('t:Q', title=None),
    alt.Y('value:Q'),
    alt.Color('model')
    ).properties(width=width, height=height).interactive()
c

alt.Chart(...)

In [6]:
np.log(100)

4.605170185988092

In [8]:
model_evidence = dict()
for m in ['m'+str(i) for i in model_nums]:
    model_evidence[m] = np.sum(bf[m][30:])

model_evidence

{'m6': -270498.91701128986,
 'm9': -274869.4968816739,
 'm10': -274082.96219079744,
 'm11': -272528.92134763754}

## Scoring Rules


In [38]:
# load existing results
scrl = dict()
for model in model_nums:
    scrl['m'+str(model)] = np.cumsum(load_obj('scoring_rule', log_dirs['m'+str(model)])[31:])


In [39]:
scrldf = pd.DataFrame.from_dict(scrl)

scrldf['t']  = np.arange(len(scrldf))
for i in [9, 10,11]:
    scrldf['m'+str(i)+'-m6'] =(scrldf['m'+str(i)] -  scrldf['m6'])
scrldf.drop(['m6', 'm9', 'm10', 'm11'], axis=1, inplace=True)
scrldf = scrldf.melt(id_vars='t', var_name='model')

# scrldf['cval'] = scrldf.value.cumsum()

In [40]:
width = 600
height = 400
c = alt.Chart(scrldf).mark_line(
    strokeWidth = 1,
    ).encode(
    alt.X('t:Q', title=None),
    alt.Y('value:Q'),
    alt.Color('model')
    ).properties(width=width, height=height).interactive()
c

alt.Chart(...)